In [1]:
!pip install spotipy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os

In [3]:
os.environ['SPOTIPY_CLIENT_ID'] = '7c019efe64ad4f898c264803996b4863'
os.environ['SPOTIPY_CLIENT_SECRET'] = '261ba5610d1a4376924fa21d08468677'
os.environ['SPOTIPY_REDIRECT_URI'] = 'https://localhost:8888/callback'

In [4]:
#Create Spotify object
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [5]:
def colect(playlist_URI):
  pl_id = sp.playlist(playlist_URI)['id']
  pl_name = sp.playlist(playlist_URI)['name']
  pl = sp.playlist_tracks(playlist_URI)['items']
  items = []
  one_item = []
  removal = ['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
  
  for item in pl:
    one_item.clear()
    one_item.append(item['track']['id'])
    one_item.append(item['track']['name'])
    one_item.append(item['track']['artists'][0]['id']) #tem muitos aaaaaa
    one_item.append(item['track']['artists'][0]['name'])
    one_item.append(item['track']['album']['id'])
    one_item.append(item['track']['duration_ms'])
    one_item.append(item['track']['album']['release_date'])
    one_item.append(item['track']['popularity'])
    af = sp.audio_features([item['track']['id']])[0]
    [af.pop(key) for key in removal]
    for key in af:
      one_item.append(af[key])
    one_item.append((item['track']['duration_ms'])/60000)
    one_item.append(pl_id)
    one_item.append(pl_name)
    release_year = item['track']['album']['release_date'][0:4]
    one_item.append(release_year)
    items.append(list(one_item))

  db = pd.DataFrame(items, columns=['track_id', 'track_name', 'artist_id', 'artist_name', 'album_id', 'duration', 'release_date', 'popularity', 'danceability', 'energy', 'key', 'loudness' ,'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_mins', 'playlist_id', 'playlist_name', 'release_year'])
  return db


In [6]:
def cleaning(db, nome):
  print("Número de registros no dataset:", len(db))
  db.dropna(inplace=True)
  db.reset_index(drop=True, inplace=True)

  print("Número de registros no dataset após remover as linhas com dados faltantes:", len(db))
  print(db.columns)
  duplicates = db[db.duplicated(keep='last')]

  print("Número de registros duplicados:", len(duplicates))
  print(duplicates)
  db.drop_duplicates(inplace=True)
  db.reset_index(drop=True, inplace=True)

  print("Número de registros sem as duplicatas:", len(db))
  columns = ['track_name', 'artist_name', 'album_id', 'artist_id', 'playlist_id', 'playlist_name', 'release_date']
  numeric_columns = db.drop(columns, axis=1)
  numeric_columns.head()
  db.to_csv('../data/{}.csv'.format(nome), index= False)
  numeric_columns.to_csv('../data/{}_numeric.csv'.format(nome), index= False)


In [7]:
playlist_link = "https://open.spotify.com/playlist/5kmc1DMbogYqQ5BrfL4IrG?si=p7bRoTh2Rg-UlK_ob-LC-w&pt=ca68703f4dc55566be1b2190aea7bfa4"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
testdb = colect(playlist_URI)
testdb.head()
cleaning(testdb, "Teste")

,track_id,track_name,artist_id,artist_name,album_id,duration,release_date,popularity,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,playlist_id,playlist_name,release_year
0,4WhZCARK6jg6XlzycNhewV,Metal Slug,28SM1PdeOuqBvbv9VMPg8S,Dj Russo,763WhUKwWZ3HFEWA8fbyvY,230675,2019-02-13,0,0.785,0.725,...,0.0299,0.0184,0.712000,0.0624,0.191,128.030,3.844583,5kmc1DMbogYqQ5BrfL4IrG,nao populares,2019
1,5020RLdSVxK2aAyuFbijbl,Just To Be In Love - Original Mix,28SM1PdeOuqBvbv9VMPg8S,Dj Russo,05DIy7IupMvHxwY6pJKlV2,186008,2014-06-29,2,0.524,0.798,...,0.0903,0.0514,0.290000,0.2390,0.310,140.029,3.100133,5kmc1DMbogYqQ5BrfL4IrG,nao populares,2014
2,7MfyfHXlB6SihbYD81f3AQ,Pink Solidiers,28SM1PdeOuqBvbv9VMPg8S,Dj Russo,63j8WgtvVz1y2XwFpHmYMg,171416,2021-12-30,2,0.946,0.868,...,0.1260,0.0127,0.052700,0.0372,0.128,128.010,2.856933,5kmc1DMbogYqQ5BrfL4IrG,nao populares,2021
3,45AEAdqkbzYLZbQrpK5QLX,Bye [Hopely & ReCharged Remix],28SM1PdeOuqBvbv9VMPg8S,Dj Russo,22qPfMosTleMTIITZDZz7m,278087,2020-07-20,1,0.676,0.910,...,0.1310,0.0055,0.003330,0.3720,0.307,133.861,4.634783,5kmc1DMbogYqQ5BrfL4IrG,nao populares,2020
4,6xIc7giDfoUiQkJsJ4iMfc,Taranteiro - DJ Russo Dub-House Remix 2018,2kduyjlz8uR5eMRTyrpSk1,Doctor Keos,2rUNRzg8h9nAuyPQzEIsMv,261698,2018-03-02,1,0.679,0.607,...,0.0811,0.0108,0.000158,0.0251,0.360,132.024,4.361633,5kmc1DMbogYqQ5BrfL4IrG,nao populares,2018


In [7]:
#Para coletar uma playlist faça:
# playlist_link = Link da playlist aqui!
# playlist_URI = playlist_link.split("/")[-1].split("?")[0]
# db = colect(playlist_URI)
# db.head()
# cleaning(testdb, Nome da playlist aqui!)